In [3]:
reactor = 1 #PWR = 1, BWR = 2

In [37]:
import numpy as np
import numpy.linalg as la
import scipy as sci
import matplotlib.pyplot as plt
from pyXSteam.XSteam import XSteam as xs

# givens
if reactor == 1:
    # geometry
    height = 4
    d_rod = 0.95 / 100
    d_fuel = 0.82 / 100
    t_gap = 0.006 / 100
    pitch = 1.26 / 100

    # heat transfer
    k_gap = 0.25
    k_fuel = 3.6
    k_clad = 21.5
    qp = 430*100

    # flow params
    g = 4000
    p_z0 = 15
    tf_z0 = 277 + 273.15

    # viscocities
    mu = 97.575e-6
    mu_l = 69.403e-6
    mu_v = 22.716e-6
    
if reactor == 2:
    # geometry
    height = 4.1
    d_rod = 1.227 / 100 
    d_fuel = 1.04/100
    t_gap= 0.010/100
    pitch = 1.62/100

    # heat transfer
    k_gap = 0.25
    k_fuel = 3.6
    k_clad = 21.5
    qp = 605*100

    # flow params
    g = 2350
    p_z0 = 7.5
    tf_z0 = 272 + 273.15
    
    # viscocities
    mu = 97.352e-6
    mu_l = 89.451e-6
    mu_v = 11.656e-6

st = XSteam(XSteam.UNIT_SYSTEM_BARE)
z = np.linspace(0,height,1000)


# calculated geometry
a_fluid = pitch**2 - np.pi / 4 * d_rod**2
xi = np.pi * d_rod
d_h = 4 * a_fluid / xi

cp = st.Cp_pt(p0,tf_z0) * 1e3
k = st.tc_pt(p0,tf_z0)


# functions
def heat_qp(z): return qp * np.sin(z * np.pi / height)
def heat_qpp(z): return qp(z) / xi

def hf(p): return st.hL_p(p) * 1e3
def hg(p): return st.hV_p(p) * 1e3
def hfg(p): return (hg(p) - hf(p))

def dhfdp(z): return (7.11762e-16*(z)**2-1.85296e-8*(z)+1.52758e-1)
def dhfgdp(z): return (-7.7343e-16*(z)**2+1.65472e-8*(z)+1.4331e-1)

def tsat(p): return st.tsat_p(p)
def pratio(p): return st.rhoV_p(p) / st.rhoL_p(p)
def alpha(chi_e, p): return (1 + (chi_e**(-1) - 1) * pratio(p))**(-1)


# dimensionless groups
re = g * d_h / mu
pr = cp * mu / k

if re > 1e4: nu = 0.023 * re**0.8 * pr**0.4
if re < 2100: nu = 4.36


# parameters
h = nu * k / d_h
f = 0.316 * re**(-0.25)

rho_fg = st.rhoL_p(p0) - st.rhoV_p(p0)
rho_l = st.rhoL_p(p0)
rho_v = st.rhoV_p(p0)

nu_l = st.vL_p(p_z0)
nu_g = st.vV_p(p_z0)
nu_fg = nu_g - nu_l


# solving for the fluid
def solve_fluid(ms):
    zs = np.linspace(0, height, ms)
    
    ps = [p_z0 * 1e6]
    chi_es = [cp * (tf_z0 - tsat(p_z0)) / hfg(p_z0)]
    chis = [0]
    tfs = [tf_z0]
    tsats = [tsat(p_z0)]
    
    for i in range(1, len(zs)):
        dz = z[i]- z[i-1]

        p0 = ps[i - 1]
        chi_e0 = chi_es[i - 1]
        tf0 = tfs[i - 1]
        tsat0 = tsats[i - 1]

        if chi_e0 <= 0: chi0 = 0
        if chi_e0 > 0 and chi_e0 < 1: chi0 = chi_e0

        rho_m = 1 / (nu_l + chi0 * nu_fg)

        # pressure
        if chi_e0 <= 0:
            dp = 9.81 / nu_l + f * g**2 * nu_l * xi / 2 / a_fluid
        if chi_e0 > 0 and chi_e0 < 1:
            mu_m = (chi_e0 / mu_v + (1 - chi_e0) / mu_l)**(-1)
            ff = 0.046 * re**(-0.2) * (mu_l / mu_m)**(-0.2)
            
            tp1 = xi / a_fluid * ff * g**2 * (nu_l + chi0 * nu_fg) / 2
            tp2 = rho_m * 9.81
            tp3 = g * heat_qp(zs[i]) * nu_fg / a_fluid / hfg(p0 * 1e-6)
        
            bp1 = dhfdp(p0) + chi_e0 * dhfgdp(p0)
            bp2 = 1 - g**2 * nu_fg / hfg(p0 * 1e-6) * bp1
            
            dp = (tp1 + tp2 + tp3) / bp2
        
        p1 = p0 - dp * dz
        dpdz = (p1 - p0)/dz
        
        # equlibrium quality
        chi_ec1 = dhfdp(p0) * dpdz + chi_e0 * dhfgdp(p0) * dpdz
        chi_ec2 = heat_qp(zs[i]) / a_fluid / g / hfg(p1 * 1e-6) - chi_ec1 / hfg(p1 * 1e-6)
        chi_e1 = chi_e0 + chi_ec2 * dz

        # vapor quality
        if chi_e1 <= 0: chi1 = 0
        if chi_e1 > 0: chi1 = chi_e1
        
        # fluid temp
        if chi_e1 <= 0: tf1 = hfg(p1 * 1e-6) / cp * chi_e1 + tsat(p1 * 1e-6)
        if chi_e1 > 0: tf1 = tsat(p1 * 1e-6)
        
        tsats.append(tsat(p1 * 1e-6))    
        ps.append(p1)
        chi_es.append(chi_e1)
        chis.append(chi1)
        tfs.append(tf1)

    return tsats, ps, chi_es, chis, tfs, zs

tsats, ps, chi_es, chis, tfs, zs = solve_fluid(1000)

In [49]:
# suface temp
tsats = np.array(tsats)
ps = np.array(ps)
chi_es = np.array(chi_es)
chis = np.array(chis)
tfs = np.array(tfs)

qps = qp * np.sin(zs * np.pi / height)
tws = np.zeros_like(tsats)

for i in range(len(zs)):
    twall = qps[i] / xi / height + tfs[i]
    if twall <= tsats[i]:
        tws[i] = twall
    else:
        F = (1 + chis[i] * pr * (rho_l / rho_v - 1))**(0.35)
        S = 1 / (1 + 0.055 * F**0.1 * re**0.16)
        
        qpp_ = qps[i] / xi
        prat = ps[i] * 1e-6 / 22.064

        hNB = 55 * prat**(0.12) * 18**(-0.5) * qpp_**(2/3) * (-np.log10(prat))**(-0.55)

        # quadratic eq coeffs
        a = F**2 * h**2 + S**2 * hNB**2
        b = -2 * (F**2 * h**2 * tfs[i] + S**2 * hNB**2 * tsats[i])
        c = -qpp_**2 + F**2 * h**2 * tfs[i]**2 + S**2 * hNB**2 * tsats[i]**2

        tws[i] = (-b + np.sqrt(b**2 - 4 * a * c)) / 2 / a

# radial comps
r_f = d_fuel /2
r_ci = d_fuel /2 + t_gap
r_co = d_rod / 2
a_ci = 2 * r_ci * np.pi
a_fluid = 2 * r_f * np.pi
a_xf = r_f**2 * np.pi
        
#fuel integration constants
C1 = 0
C3 = -k_fuel / k_gap * (heat_qp(z) * r_f**2 / (2 * k_fuel * a_xf))
C5 = k_gap / k_clad * C3
C6 = tws - C5 * np.log(r_co)
C4 = C5 * np.log(r_ci) - C3 * np.log(r_ci) + C6
C2 = C3 * np.log(r_f)+ C4 + heat_qp(z) * (r_f)**2 / (4 * k_fuel * a_xf)

def T_fuel(r): return -heat_qp(z) * r**2 / (4 * k_fuel * a_xf) + C2
def T_gap(r): return C3 * np.log(r) + C4
def T_clad(x): return C5 * np.log(r) + C6

tck = np.zeros(zs.size)
for i in range(zs.size):
    tck[i] = 273.15
tw_c = tws - tck

/tmp/ipykernel_973270/1245237139.py:29: RuntimeWarning: invalid value encountered in sqrt
  tws[i] = (-b + np.sqrt(b**2 - 4 * a * c)) / 2 / a
